In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import gensim
import gensim.downloader as api
from gensim.models.fasttext import load_facebook_model
import random
from difflib import SequenceMatcher
from scipy import stats
import sacrebleu
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os
import spacy
import re
import edist.sed as sed
from sacremoses import MosesTokenizer, MosesDetokenizer
from sklearn.metrics import matthews_corrcoef

import sys  
sys.path.insert(0, '../')

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


### Align different translations, find which words affect the translation of which words


**Note**: can use [sequence alignments](https://stackoverflow.com/questions/5055839/word-level-edit-distance-of-a-sentence) to align the sentences on the target side only. ([code](https://gist.github.com/slowkow/06c6dba9180d013dfd82bec217d22eb5))

Pros: could be easier than SRC-TGT alignment

Cons: in the case where more output different sentence structure yet same meaning. <br>
E.g., "Today I think the cat is nice" -- "I think the cat is nice today"
SRC-TGT alignment would probably see these as the same, but edit distance cannot, bc it only has del, insert, substitute operations.


Provided in functions `analyse_single_sentence_single_perturbed_word()` and  `analyse_single_sentence`

# Quality analysis

- Have to use WMT21 data, bc models for 2021 is available. Also they have clear evaluation script
- Have to do some manual fix so that the translation tokenization match completely with the tokenization of the labeled data

### Word-level

- A translated word is uncertain if changing other words in the SRC sentence affect its translations. The assumption is the the translation of one word should only depends on a few others word, but not too many.
    - E.g., My mother, who had a difficult childhood, is a great doctor.
    - The gender form of "doctor" should only change if we change the word "mother".

- Hyperparam: The number of SRC words that effect the translations of the target word
    - Currently: If a translated word has 3 or more effecting SRC word, mark as "BAD"

- Metrics: Matthews correlation coefficient 
    "It takes into account true and false positives and negatives and is generally regarded as a balanced measure which can be used even if the classes are of very different sizes. The MCC is in essence a correlation coefficient value between -1 and +1. A coefficient of +1 represents a perfect prediction, 0 an average random prediction and -1 an inverse prediction."
    
- Unsupervised baseline: since our approach is based on the uncertainty of a translated word, the baseline could be the word-level log probablity generated by the NMT model itself. (i.e., word with low certainty --> BAD)

Current score: 0.267

Score of WMT21 shared task: baseline 0.370, best 0.510

WMT21 baseline: multilingual transformer-based Predictor-Estimator approach for both sentence level and word level


**How to do hyperparameter tuning if our goal is not to use the training data? Or just using the dev set is oke?**



### Baseline
Uses word-level log probablity generated by the NMT model itself. (i.e., word with low certainty --> BAD).

Note that the NMT model output subwords and subword log probs, so for a word `A=a1a2` made of subwords `a1`, `a2`:

prob(a1a2 | things before it, SRC) = prob(a1 | things before it, SRC) *  prob(a2 | a1, things before it, SRC) 

$$log(prob(A)) = log(prob(a1)) + log(prob(a2))$$




Some times it output subwords that contains 2 tokens (i.e., the opposite of the above case). This does not happen very often

Then we approximate:
$$log(prob(a1)) = log(prob(a2)) = log(prob(A))/2$$

**Is this correct???**



Current best hyperparams setting on dev en-de:

Our approach: hyperparams: 
- Perturbing [allTokens, allWords, allContentWords]
- Word threshold: [1,2,3,4]

Nmt word probas:
- Threshold 0.5

#### Word-level QE on translation

- Our perturbation approach:
    - **AllTokens, word threshold 3: 0.2716**
    - AllWords, word threshold 3: 0.2647
    - Content, word threshold 2: 0.2667
- Use nmt model word probas:
    - threshold 0.5: 0.2590
    
    
#### Word-level QE on SRC
- Our perturbation approach:
    - **AllTokens, word threshold 3: 0.1308**
    - AllWords, word threshold 3: 0.1275
    - Content, word threshold 2: 0.1285
- Use nmt model word probas:
    - threshold 0.5: 0.2053
    
--> Worse than using nmt model word probas

In [ ]:
from align_and_analyse_ambiguous_trans import analyse_single_sentence_single_perturbed_word, analyse_single_sentence, align_translations
from quality_estimation import nr_effecting_src_words_eval, nmt_log_prob_eval, flatten_list, load_gold_labels, replace_unknown

dataset = 'WMT21_DA_dev'
data_root_path = '../data'
src_lang = 'en'
tgt_lang = 'de'
effecting_words_threshold = 3
nmt_log_prob_threshold = 0.5
task = 'trans_word_level_eval'
perturbed_trans_df_path = f'../analyse_output/{dataset}_{src_lang}2{tgt_lang}_MultiplePerSentence_allTokens/analyse_{dataset}_{src_lang}2{tgt_lang}_MultiplePerSentence_allTokens.pkl'

perturb_based_pred = nr_effecting_src_words_eval(perturbed_trans_df_path, effecting_words_threshold, task)
nmt_prob_based_pred = nmt_log_prob_eval(dataset, data_root_path, src_lang, tgt_lang, nmt_log_prob_threshold, perturbed_trans_df_path, task)
gold_labels = load_gold_labels(dataset, data_root_path, src_lang, tgt_lang, task)




In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score ,accuracy_score

labels = ['OK', 'BAD']
recall = recall_score(flatten_list(gold_labels), 
                                      flatten_list(nmt_prob_based_pred), labels=labels, pos_label='BAD')
precision = precision_score(flatten_list(gold_labels), 
                                      flatten_list(nmt_prob_based_pred), labels=labels, pos_label='BAD')

print(f"Recall wrt BAD: {recall}")
print(f"Precision wrt BAD: {precision}")
print(f"Matthews_corrcoef: {matthews_corrcoef(flatten_list(gold_labels), flatten_list(nmt_prob_based_pred))}")

print(f"Percentage of unknown labels: {accuracy_score(flatten_list(nmt_prob_based_pred), ['unknown']*len(flatten_list(gold_labels)))*100}")

dist = ConfusionMatrixDisplay(confusion_matrix(flatten_list(gold_labels), 
                                               flatten_list(nmt_prob_based_pred), labels=labels),
                             display_labels=labels)
dist.plot()



In [ ]:
recall = recall_score(flatten_list(gold_labels), 
                                      flatten_list(perturb_based_pred), labels=labels, pos_label='BAD')
precision = precision_score(flatten_list(gold_labels), 
                                      flatten_list(perturb_based_pred), labels=labels, pos_label='BAD')

print(f"Recall wrt BAD: {recall}")
print(f"Precision wrt BAD: {precision}")
print(f"Matthews_corrcoef: {matthews_corrcoef(flatten_list(gold_labels), flatten_list(perturb_based_pred))}")
print(f"Percentage of unknown labels: {accuracy_score(flatten_list(perturb_based_pred), ['unknown']*len(flatten_list(gold_labels)))*100}")


dist = ConfusionMatrixDisplay(confusion_matrix(flatten_list(gold_labels), 
                                               flatten_list(perturb_based_pred), labels=labels),
                             display_labels=labels)
dist.plot()

Looking into the percentage of same predictions between `perturb_based_pred` and `nmt_prob_based_pred` to see if our approach is actually any different.

0.6835, ok so not too much

In [ ]:
accuracy_score(flatten_list(gold_labels), flatten_list(perturb_based_pred))

### Analyse some specific samples

In [ ]:
perturbed_trans_df = pd.read_pickle(perturbed_trans_df_path)


In [ ]:
sentence_idx = 3
sentence_df = perturbed_trans_df[perturbed_trans_df['SRC_original_idx'] == sentence_idx]
original_SRC = sentence_df['SRC'].values[0]
original_translation = sentence_df['SRC-Trans'].values[0]
tok_original_translation = sentence_df['tokenized_SRC-Trans'].values[0]
tok_original_SRC = sentence_df['tokenized_SRC'].values[0]

my_pred = np.array(perturb_based_pred[sentence_idx])
nmt_pred = np.array(nmt_prob_based_pred[sentence_idx])
gold = np.array(gold_labels[sentence_idx])


# BAD words correctly predicted by perturb_based_pred
# bad_word_indices = np.nonzero(np.logical_and(my_pred=='BAD', gold=='BAD'))

# # OK words predicted by perturb_based_pred as BAD
bad_word_indices = np.nonzero(np.logical_and(my_pred=='BAD', gold=='OK'))


# # BAD words correctly predicted by perturb_based_pred, but not by nmt_prob_based_pred
# bad_word_indices = np.nonzero(np.logical_and(np.logical_and(my_pred=='BAD', gold=='OK'), gold=='BAD'))

print(len(bad_word_indices[0]))
word_idx = bad_word_indices[0][1]


if task == 'src_word_level_eval':
    word = tok_original_SRC[word_idx]
    align_type = "src-trans"
elif task == 'trans_word_level_eval':
    word = tok_original_translation[word_idx]
    align_type = "trans-only"
else:
    raise RuntimeError()



In [ ]:
import pprint


print(f"Original SRC sentence:\n{original_SRC}")
print(f"Original trans:\n{original_translation}")
print(f"BAD word: {word}")
print()

pprint.pprint(analyse_single_sentence(sentence_df, align_type=align_type, return_word_index=True))


In [ ]:
original_word = 'Churchyard'


sentence_single_perturbed_word_df = sentence_df[sentence_df['original_word'] == original_word]


pprint.pprint(analyse_single_sentence_single_perturbed_word(sentence_single_perturbed_word_df, align_type=align_type))
align_translations(sentence_single_perturbed_word_df, align_type=align_type)


## Sentence level

Approximations:
- Negative corr with DA:
    - Changes edit distance 
    - Changes edit distance / length
    - Changes spread
    - Changes spread / length
    - Number of BAD tokens
   
Metrics: Pearson correlation coefficient: "Correlations of -1 or +1 imply an exact linear relationship"

DA scores:
Scores: highest 0.18
WMT21 scores: baseline 0.403, best 0.584


HTER scores: 
Scores: highest 0.28
WMT21 scores: baseline 0.529, best 0.653

**Can try to apply some function to the prediction, but then that's not unsupervised anymore**

**Can use as feature for QE model, but again not unsupervised anymore**

In [ ]:
from scipy.stats import zscore
from scipy.stats import pearsonr, spearmanr


approximations = output[
    ["SRC_original_idx", 
     "Trans-edit_distance", 
     "#TransChanges/SentenceLength",
     "ChangesSpread",
     "ChangesSpread/SentenceLength"
    ]
].groupby("SRC_original_idx").mean()

approximations['word_level_agg'] = [x.count('BAD') for x in word_tag]


for col in approximations.columns:
    # Normalize the apporximations, invert the sign
    approximations[col] = -approximations[col]
    approximations[col] = zscore(approximations[col].values)
    
    
approximations.head()

In [ ]:
# trans_analysed_file = "analyse_WMT22_MQM_en2de.pkl"
# output = pd.read_pickle(trans_analysed_file)

Gold labels:

In [ ]:
if 'WMT21_DA' in trans_analysed_file:
    with open("../data/wmt-qe-2021-data/en-de-test21/goldlabels/test21.hter", 'r') as f:
        da_scores = f.readlines()
        da_scores = [float(da_score.replace('\n', '')) for da_score in da_scores]
    gold_lables = da_scores
elif 'WMT22_MQM' in trans_analysed_file:
    with open("../data/wmt-qe-2022-data/test_data-gold_labels/task1_mqm/en-de/test.2022.en-de.mqm_z_score", 'r') as f:
        mqm_scores = f.readlines()
        mqm_scores = [float(mqm_score.replace('\n', '')) for mqm_score in mqm_scores]
    gold_lables = mqm_scores

    

Evaluation on gold labels:

In [ ]:
for col in approximations.columns:
    print(f"-----------------{col}-----------------")
    print(pearsonr(gold_lables, approximations[col].values))


In [ ]:
for col in approximations.columns:
    plot_df = pd.DataFrame({'true': gold_lables, 'pred': approximations[col].values})
#     plot_df = plot_df.sort_values('pred')
    
    X = plot_df['pred']
    Y = plot_df['true']
    
    plt.figure()
    hist = plt.hist(Y, bins=20)
    bin_boundaries = hist[1]
    
#     # Remove bins with too few samples
#     cut_point = 99999
#     for i, value in enumerate(hist[0]):
#         if value < 5:
#             cut_point = i
#             break

#     bin_boundaries = bin_boundaries[:cut_point]
    
    
    plt.figure()
    x_plot = [(bin_boundaries[i] + bin_boundaries[i+1])/2 for i in range(0, len(bin_boundaries)-1)]
    y_plot = [stats.trim_mean(Y[(bin_boundaries[i] < X) & (X < bin_boundaries[i+1])], 0.1) for i in range(0, len(bin_boundaries)-1)]
    plt.plot(x_plot, y_plot)
    plt.xlabel(col)
    plt.ylabel('gold_lables')


    
    

In [ ]:
trans_analysed_file = "tmp_storages/analyse_WMT21_DA_dev_en2de_MultiplePerSentence_allWords.pkl"
output = pd.read_pickle(trans_analysed_file)

# original_src_errornous_idxs = [17, 122, 306, 817, 908, 940]


def fix_tokenization(tokenized_sentence, dataset):
    # Only for WMT21_DA_en2de data
    # Some of the sentences is tokenized differently in the labeled data. I.e., the last dot is not tokenized
    # Fix in order to syncronize with the labeled data
    if tokenized_sentence[-1] != '.':
        str_sentence = ' '.join(tokenized_sentence)
        str_sentence = str_sentence[:-1] + ' .'
        return str_sentence.split()
    else:
        return tokenized_sentence

# output['tokenized_SRC'] = output.apply(
#                 lambda x: fix_tokenization(
#                     x['tokenized_SRC']
#                 ) if x['SRC_original_idx'] in original_src_errornous_idxs else x['tokenized_SRC'],
#                 axis=1
#             )


toks = output.groupby('SRC_original_idx').first()['tokenized_SRC'].tolist()
toks = [' '.join(tok) for tok in toks]

with open('/Users/tuanh/Desktop/tmp.txt', 'w') as f:
    for x in toks:
        f.writelines(x + '\n')
